# Leaflet cluster map of talk locations

Assuming you are working in a Linux or Windows Subsystem for Linux environment, you may need to install some dependencies. Assuming a clean installation, the following will be needed:

```bash
sudo apt install jupyter
sudo apt install python3-pip
pip install python-frontmatter getorg --upgrade
```

After which you can run this from the `_talks/` directory, via:

```bash
 jupyter nbconvert --to notebook --execute talkmap.ipynb --output talkmap_out.ipynb
```
 
The `_talks/` directory contains `.md` files of all your talks. This scrapes the location YAML field from each `.md` file, geolocates it with `geopy/Nominatim`, and uses the `getorg` library to output data, HTML, and Javascript for a standalone cluster map.

In [1]:
# Start by installing the dependencies
!pip install python-frontmatter getorg --upgrade
import frontmatter
import glob
import getorg
from geopy import Nominatim
from geopy.exc import GeocoderTimedOut

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 7.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [getorg]2m7/9 [pygithub]phy]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Iywidgets and ipyleaflet support disabled. You must be in a Jupyter notebook to use this feature.
Error raised:
No module named 'ipyleaflet'
Check that you have enabled ipyleaflet in Jupyter with:
    jupyter nbextension enable --py ipyleaflet


In [14]:
# Collect the Markdown files
g = glob.glob("_talks/*.md")

In [15]:
# Set the default timeout, in seconds
TIMEOUT = 5

# Prepare to geolocate
geocoder = Nominatim(user_agent="academicpages.github.io")
location_dict = {}
location = ""
permalink = ""
title = ""

In the event that this times out with an error, double check to make sure that the location is can be properly geolocated.

注意! 東京大学駒場は、目黒区とちょっと渋谷区にまたがっているが、OpenStreetMap上で渋谷区に登録されているため、「東京都 東京大学 駒場Iキャンパス」としなければならない。

In [16]:
# Perform geolocation
for file in g:
    # Read the file
    data = frontmatter.load(file)
    data = data.to_dict()

    # Press on if the location is not present
    if 'location' not in data or data['location'].strip() == "オンライン" or data['location'].strip() == "Online":
        continue

    # Prepare the description
    title = data['title'].strip()
    venue = data['venue'].strip()
    location = data['location'].strip()
    description = f"{title}<br />{venue}; {location}"

    # Geocode the location and report the status
    try:
        location_dict[description] = geocoder.geocode(location, timeout=TIMEOUT)
        print(description, location_dict[description])
    except ValueError as ex:
        print(f"Error: geocode failed on input {location} with message {ex}")
    except GeocoderTimedOut as ex:
        print(f"Error: geocode timed out on input {location} with message {ex}")
    except Exception as ex:
        print(f"An unhandled exception occurred while processing input {location} with message {ex}")

ツリー法を応用した多成分ダストの合体成長計算アルゴリズム<br />2025年度 第55回天文・天体物理若手夏の学校; 長野県千曲市 戸倉上山田温泉 ホテル圓山荘 ホテル圓山荘, 中央通り, 上山田温泉二丁目, 千曲市, 長野県, 389-0821, 日本
ツリー法を応用した多成分ダストの合体成長計算アルゴリズム<br />日本惑星科学会2025年秋季講演会; 東京都 東京大学 駒場Iキャンパス 東京大学 駒場Ⅰキャンパス, 山手通り, 松濤二丁目, 松濤, 渋谷区, 東京都, 150-0046, 日本
ツリー法を応用した多成分ダストの合体成長計算アルゴリズム<br />日本天文学会2025年秋季年会; 山口県下関市 海峡メッセ下関 海峡メッセ下関, 豊前田町三丁目, 下関市, 山口県, 日本


In [17]:
# Save the map
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap", hashed_usernames=False)

'Written map to talkmap/'